## Activity - 1

* Organizing the Images into Different Classes.

In [1]:
import os
import shutil
import sys
import pandas as pd

In [2]:
dataset_dir = '/content/drive/MyDrive/al_course/content/train'
labels = pd.read_csv('/content/drive/MyDrive/al_course/content/labels.csv')

In [3]:
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [4]:
import os
def make_dir(x):
 if os.path.exists(x) == False:
     os.makedirs(x)
base_dir = '/content/drive/MyDrive/al_course/content/subset'
make_dir(base_dir)

In [5]:
train_dir = os.path.join(base_dir, 'train')
make_dir(base_dir)

breeds = labels.breed.unique()

In [ ]:
breeds = labels.breed.unique()
for breed in breeds:
    #Make folder for each breed
    _ = os.path.join(train_dir, breed)
    make_dir(_)

    #Copy images to the crresponsing folder
    images = labels[labels.breed == breed]['id']
    for image in images:
        source = os.path.join(dataset_dir, f'{image}.jpg')
        destination = os.path.join(train_dir, breed, f'{image}.jpg')
        try:
          shutil.copyfile(source,destination)
        except:
          pass

## Activity - 2

* Import the ImageDataGenerator library.

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Activity - 3

* Configure ImageDataGenerator class

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

## Activity - 4

* Apply ImageDataGenerator functionality to Trainset and Test set

In [3]:
datagen = ImageDataGenerator()
generator = datagen.flow_from_directory(
    "/content/drive/MyDrive/al_course/new_dataset/dog-breeds",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 541 images belonging to 8 classes.


# Model Building

## Activity - 1
* Importing the Model Building Libraries

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import softmax
from keras.api._v2.keras import activations

## Activity - 2
* Importing the  VGG19 model

In [5]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

## Activity - 3
* Initializing the model

In [6]:
Image_size=(224, 224, 3)

In [7]:
sol=VGG19(input_shape=Image_size, weights='imagenet', include_top=False)

80134624/80134624 [==============================] - 4s 0us/step


In [8]:
for i in sol.layers:
  i.trainable = False

In [9]:
y=Flatten()(sol.output)

## Activity - 4
* Adding Fully connected Layers

In [15]:
from keras.api._v2.keras import activations
final = Dense(8, activation='softmax')(y)

In [16]:
sol.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

## Activity - 5
* Configure The Learning Process

In [17]:
sol = Model(inputs=sol.input,outputs=final)

In [18]:
sol.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['Accuracy'])

## Activity - 6
* Train the Model

In [19]:
sol.fit(generator, epochs=6, batch_size=32)

Epoch 1/6
17/17 [==============================] - 364s 23s/step - loss: 91.2607 - Accuracy: 0.1941
Epoch 2/6
17/17 [==============================] - 7s 415ms/step - loss: 4.7653 - Accuracy: 0.8651
Epoch 3/6
17/17 [==============================] - 9s 496ms/step - loss: 1.3239 - Accuracy: 0.9519
Epoch 4/6
17/17 [==============================] - 7s 411ms/step - loss: 0.5945 - Accuracy: 0.9815
Epoch 5/6
17/17 [==============================] - 9s 492ms/step - loss: 4.5003e-06 - Accuracy: 1.0000
Epoch 6/6
17/17 [==============================] - 7s 410ms/step - loss: 0.0013 - Accuracy: 1.0000


In [20]:
sol.save("/content/drive/MyDrive/al_course/content/subset/dogbreed_vgg19.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
